This file implements creates a dataset with an implementation of matching via propensity score for more accurate controls (in contrast to the initial sample restriction).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

In [2]:
leokas = ['ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2017.dta',
          'ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2016.dta',
          'ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2015.dta',
          'ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2014.dta',
          'ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2013.dta']
leoka_17, leoka_16, leoka_15, leoka_14, leoka_13 = pd.read_stata(leokas[0]), pd.read_stata(leokas[1]), pd.read_stata(leokas[2]), pd.read_stata(leokas[3]), pd.read_stata(leokas[4])

dfs = [leoka_13, leoka_14, leoka_15, leoka_16, leoka_17]
dfs = [i.iloc[:,:-198] for i in dfs]

for j in dfs:
    print(j.shape)

(266424, 58)
(267972, 58)
(270288, 58)
(271740, 58)
(273408, 58)


In [3]:
df = pd.concat(dfs, ignore_index=True)
df.shape

(1349832, 58)

In [4]:
month_map = {'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6, 'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12}
df['month_code'] = df['month'].map(month_map) + (df['year'] - 2013)*12
df.head(10)

,ori,agency_name,state,state_abb,number_of_months_reported,year,month,date,ori9,fips_state_code,...,assaults_with_injury_knife,assaults_with_injury_oth_weap,assaults_with_injury_unarmed,assaults_with_injury_total,assaults_no_injury_gun,assaults_no_injury_knife,assaults_no_injury_oth_weap,assaults_no_injury_unarmed,assaults_no_injury_total,month_code
0,AK00101,anchorage,alaska,AK,12,2013,january,2013-01-01,AK0010100,02,...,0,0,0,0,1,1,1,27,30,1
1,AK00101,anchorage,alaska,AK,12,2013,february,2013-02-01,AK0010100,02,...,0,1,1,2,0,0,9,13,22,2
2,AK00101,anchorage,alaska,AK,12,2013,march,2013-03-01,AK0010100,02,...,0,1,2,3,0,0,6,19,25,3
3,AK00101,anchorage,alaska,AK,12,2013,april,2013-04-01,AK0010100,02,...,0,0,0,0,2,0,2,15,19,4
4,AK00101,anchorage,alaska,AK,12,2013,may,2013-05-01,AK0010100,02,...,0,0,0,0,0,0,6,17,23,5
5,AK00101,anchorage,alaska,AK,12,2013,june,2013-06-01,AK0010100,02,...,0,0,0,0,2,2,3,17,24,6
6,AK00101,anchorage,alaska,AK,12,2013,july,2013-07-01,AK0010100,02,...,0,2,1,3,0,1,0,15,16,7
7,AK00101,anchorage,alaska,AK,12,2013,august,2013-08-01,AK0010100,02,...,0,0,0,0,0,0,3,13,16,8
8,AK00101,anchorage,alaska,AK,12,2013,september,2013-09-01,AK0010100,02,...,0,0,0,0,0,0,1,11,12,9
9,AK00101,anchorage,alaska,AK,12,2013,october,2013-10-01,AK0010100,02,...,0,0,3,3,2,0,3,14,19,10


In [5]:
#start by making a dataset of treated municipalities for balance calculation
df['total_assaults'] = df['assaults_no_injury_total'] + df['assaults_with_injury_total']
officer_assaulted = (df['total_assaults'] > 0)
df['killed_indicator'] = np.where(df['officers_killed_total'] > 0, 1, 0)
officer_killed = (df['killed_indicator'] == 1)

#making treated df
treated_oris = df.loc[officer_killed, 'ori'].unique()
treated_df = df[df['ori'].isin(treated_oris)]
treated_df.shape

(15564, 61)

In [6]:
#for comparison, drop the treated rows to make an untreated df
df = df[df['ori'].isin(treated_oris) == False]
df.shape

(1334268, 61)

In [7]:
#filter df further into a sample restricted df 
ori_to_keep = df.loc[officer_assaulted, 'ori'].unique()
sr_df = df[df['ori'].isin(ori_to_keep)]
treated_df.shape, sr_df.shape, df.shape, #df is untreated things, treated is anything with a killing, sr_df is sample-restricted? anything with a killing or attempted killing

((15564, 61), (518592, 61), (1334268, 61))

In [ ]:
#generate a couple of other variables of interest
for i in [sr_df, treated_df, df]:
    i['pct_female_officers'] = np.where(i['total_employees_officers'] > 0, i['female_employees_officers']/i['total_employees_officers'], 0)
    i['pct_officers_civilians'] = np.where(i['total_employees_total'] > 0, i['total_employees_civilians']/i['total_employees_total'], 0)
    i['pct_population_officers'] = np.where(i['population'] > 0, i['total_employees_total']/i['population'], 0)

In [12]:
covariates = ['population', 'pct_female_officers', 'pct_officers_civilians', 'pct_population_officers', 'total_employees_total',
              'assaults_with_injury_knife', 'assaults_with_injury_oth_weap', 'assaults_with_injury_unarmed', 'assaults_with_injury_total', 'total_assaults']

Iteration 2 of t-tests: dropping largest cities

In [9]:
top_10_pops = sorted(df.population.unique(), reverse=True)[:10]
df_filt = df[~df['population'].isin(top_10_pops)]
df_filt.shape

(1334148, 64)

In [10]:
treated_df_filt = treated_df[~treated_df['population'].isin(top_10_pops)]
sr_df_filt = sr_df[~sr_df['population'].isin(top_10_pops)]
sr_df_filt.shape, treated_df_filt.shape

((518592, 64), (15564, 64))

In [14]:
for c in covariates:
    m1, m2, m3 = np.mean(treated_df_filt[c]), np.mean(df_filt[c]), np.mean(sr_df_filt[c])
    t_12, p_12 = ttest_ind(treated_df_filt[c], df_filt[c])
    t_13, p_13 = ttest_ind(treated_df_filt[c], sr_df_filt[c])
    print(f'for {c}: overall balance means: {m1, m2} and t-stat: {t_12} and p-val: {p_12}')
    #print(f'for {c}: restricted balance means: {m1, m3} and t-stat: {t_13} and p-val: {p_13}')

for population: overall balance means: (162471.58905165768, 12308.173881758246) and t-stat: 314.05161618196587 and p-val: 0.0
for pct_female_officers: overall balance means: (0.09097004948196198, 0.055312340259872375) and t-stat: 44.99979489692858 and p-val: 0.0
for pct_officers_civilians: overall balance means: (0.2466978975325867, 0.14110424733328308) and t-stat: 71.75536697634887 and p-val: 0.0
for pct_population_officers: overall balance means: (0.0024359990098380474, 0.002223649071195938) and t-stat: 1.3750847701041446 and p-val: 0.16910539366743713
for total_employees_total: overall balance means: (521.8689282960678, 41.06803443096268) and t-stat: 206.47041340017674 and p-val: 0.0
for assaults_with_injury_knife: overall balance means: (0.0049473143150860965, 0.0004707123947268219) and t-stat: 20.751500801037373 and p-val: 1.2299481236527433e-95
for assaults_with_injury_oth_weap: overall balance means: (0.1195065535851966, 0.00685081415255279) and t-stat: 90.67637245105921 and p-v

Now aggregate dataset by municipality and take the relevant features out: population_group, longitude, latitutde, country_division, pct_female_officers, pct_officers_civilians, pct_population_officers, shift_data, covered_by; label is killed_indicator

In [27]:
len(treated_oris) #260
len(df['ori'].unique()) #22524

22524

In [10]:
#BELOW: calculate a propensity score for each observation
#sort by score, pick unique until we get to 260 un-treated

import numpy as np
import statsmodels.api as sm

In [11]:
for col in df.columns:
    if df[col].dtype == np.float64:
        df[col].fillna(df[col].mean(), inplace=True)

df.dropna(inplace=True)

In [12]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

df.dropna(inplace=True)

In [13]:
df['treated'] = df['ori'].isin(treated_oris).astype(int)

In [14]:
cols_to_convert = ['longitude', 'latitude']

for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [18]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

df.dropna(inplace=True)

In [ ]:
X = df[['longitude', 'latitude', 'pct_female_officers', 'pct_officers_civilians', 'pct_population_officers']]
X = sm.add_constant(X)
y = df['treated']

model = sm.Logit(y, X).fit()
df['pscore'] = model.predict(X)

In [35]:
print(df[['longitude', 'latitude', 'pct_female_officers', 'pct_officers_civilians', 'pct_population_officers']].dtypes)

longitude                   object
latitude                    object
pct_female_officers        float64
pct_officers_civilians     float64
pct_population_officers    float64
dtype: object


In [16]:
for col in ['longitude', 'latitude', 'pct_female_officers', 'pct_officers_civilians', 'pct_population_officers']:
    non_numeric = df[col].apply(lambda x: isinstance(x, (int, float)))
    if not non_numeric.all():
        print(f"Non-numeric values found in {col}:")
        print(df[~non_numeric][col].unique())

In [20]:
muni_pscores = df.groupby('ori').agg({'pscore': 'mean'}).reset_index()

In [21]:
muni_pscores = muni_pscores.sort_values(by=['pscore'], ascending=False)
muni_pscores

,ori,pscore
13,AK00116,2.542672e-10
4,AK00106,2.039463e-10
11,AK00113,2.002982e-10
41,AK00164,1.205798e-10
19,AK00123,6.239321e-11
...,...,...
996,CA00411,6.411106e-25
1277,CA02116,4.380987e-25
1521,CA03718,3.688684e-25
36,AK00150,5.238149e-26


In [ ]:
#treated_oris
muni_pscores = muni_pscores[muni_pscores['ori'].isin(treated_oris) == False]

#untreated_oris to keep
muni_pscores['ori']

control_oris = muni_pscores['ori'].head(260).tolist()
control_oris

treated_oris = treated_oris.tolist()
treated_oris, control_oris

In [25]:
all_oris = []

for i in control_oris:
    all_oris.append(i)

for i in treated_oris:
    all_oris.append(i)

len(all_oris)

In [44]:
mdf = df[df['ori'].isin(all_oris)]
mdf.shape

(15552, 66)

Remainder of code runs all the regular processing from original regression prep file

In [ ]:

mdf['first_pop'] = mdf.groupby('ori')['population'].transform('first')
mdf['first_employment'] = mdf.groupby('ori')['total_employees_officers'].transform('first')

In [46]:
#column transformations

#drop 0 pop rows to avoid division by 0
mdf = mdf.loc[mdf['first_pop'] > 0]

#force size mechanism construction
median_condition = mdf['first_employment'] > mdf['first_employment'].median()
trueval, falseval = 1, 0
mdf['employment_median_indicator'] = np.where(median_condition, trueval, falseval)

#proportion mechanism
mdf['employment_pop_proportion'] = mdf['first_employment']/mdf['first_pop']
prop_median_condition = mdf['employment_pop_proportion'] > mdf['employment_pop_proportion'].median()
mdf['employment_pop_prop_indicator'] = np.where(prop_median_condition, trueval, falseval)

In [47]:
mdf = mdf.sort_values(by=['ori', 'month_code'])

In [48]:
#post and pre indicators (for both shooting and implicitly killing)
for i in range(10):
    mdf[f'PreviousTreatment_{i}'] = mdf.groupby('ori')['total_assaults'].shift(i)
    mdf[f'post_{i}'] = mdf[f'PreviousTreatment_{i}'] > 0

    #don't do the idxmin thing, shouldn't matter since things are NaN where edge case shifts; drop the NaNs by removing the PreviousTreament_i columns

    mdf[f'post_{i}'] = mdf[f'post_{i}'].astype(int)
    mdf = mdf.drop(columns=[f'PreviousTreatment_{i}'])

In [49]:
#make three pre-treatment month indicators
for i in range(1, 4, 1):
    mdf[f'FutureTreatment_{i}'] = mdf.groupby('ori')['total_assaults'].shift(-1*i)
    mdf[f'pre_{i}'] = mdf[f'FutureTreatment_{i}'] > 0
    mdf[f'pre_{i}'] = mdf[f'pre_{i}'].astype(int)
    mdf = mdf.drop(columns=[f'FutureTreatment_{i}'])

In [50]:
mdf['assault_indicator'] = (mdf['total_assaults'] > 0).astype(int)

In [35]:
#bring in UCR data, combine, and run triple diff
ucrs = ['ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2017.dta',
        'ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2016.dta',
        'ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2015.dta',
        'ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2014.dta',
        'ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2013.dta']

ucr_17, ucr_16, ucr_15, ucr_14, ucr_13 = pd.read_stata(ucrs[0]), pd.read_stata(ucrs[1]), pd.read_stata(ucrs[2]), pd.read_stata(ucrs[3]), pd.read_stata(ucrs[4])


In [ ]:
ucr_dfs = [ucr_13, ucr_14, ucr_15, ucr_16, ucr_17]
ucr = pd.concat(ucr_dfs, ignore_index=True)
ucr['total_arrests'] = ucr['theft_tot_arrests'] + ucr['robbery_tot_arrests'] + ucr['rape_tot_arrests'] + ucr['murder_tot_arrests'] + ucr['mtr_veh_theft_tot_arrests'] + ucr['burglary_tot_arrests'] + ucr['arson_tot_arrests'] + ucr['agg_assault_tot_arrests']
ucr['log_total_arrests'] = np.log(ucr['total_arrests'] + 1)

In [ ]:
ucr['month_code'] = ucr['month'].map(month_map) + (ucr['year'] - 2013)*12
mdf = mdf.drop_duplicates(subset=['ori', 'month_code'])
ucr = ucr.drop_duplicates(subset=['ori', 'month_code'])

In [38]:
ucr.shape
merged = pd.merge(mdf, ucr, on=['ori', 'month_code', 'year'], how='left')
mdf.shape, ucr.shape, merged.shape

((12072, 85), (846711, 632), (12072, 714))

In [52]:
merged.to_stata('event_study_matched_2.dta')

In [39]:
#dataset creation for stage 3
overall_conditions = [(merged['post_0'] + merged['post_1'] + merged['post_2'] + merged['post_4'] + merged['post_5'] + merged['post_6'] +merged['post_7'] 
                       + merged['post_8'] + merged['post_9'] >= 1),
                      (merged['pre_1'] + merged['pre_2'] + merged['pre_3'] >= 1)]
overall_condition_values = [1, 0] #2 is default value
merged['post_overall'] = np.select(overall_conditions, overall_condition_values, default=2)

In [40]:
avg_data = merged.loc[merged['post_overall'] < 2]

In [41]:
merged.shape, avg_data.shape

((12072, 715), (0, 715))

In [42]:
merged.to_stata('event_study_matched.dta')